In [1]:
import kfp
from kfp.components import InputPath, OutputPath
from kfp import dsl
from typing import List, Tuple
from kfp.dsl import ContainerOp
from kubernetes.client.models import V1EnvVar,V1EnvVarSource, V1SecretKeySelector,V1ConfigMapKeySelector
from typing import NamedTuple

In [2]:
BASE_IMAGE = "quay.io/ntlawrence/explain-demo@sha256:aaf247201ef04f14332756a87f54b2419a2374732bc82d69030abcbd76699974"

In [3]:
def configure_tensorboard(
    mlpipeline_ui_metadata_path: OutputPath(),
    pvc_name: str,
    pvc_path: str = "",
    tensorboard_name: str = "",
) -> None:
    """
    Monitors a training job based on Tensorboard logs. 
    Logs are expected to be written to the specified subpath of the pvc
    """
    from collections import namedtuple
    import json
    from kubernetes import client, config, watch
    import logging
    import sys
    import os
    import yaml
    import textwrap
    import json
    import http

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()

    if not tensorboard_name:
        tensorboard_name="{{workflow.name}}"
        
    namespace="{{workflow.namespace}}"

    config.load_incluster_config()
    api_client = client.ApiClient()
    apps_api = client.AppsV1Api(api_client)
    custom_object_api = client.CustomObjectsApi(api_client)

    # Delete possible existing tensorboard
    try:
        custom_object_api.delete_namespaced_custom_object(
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            name=tensorboard_name,
            body=client.V1DeleteOptions()
        )
    except client.exceptions.ApiException as e:
        if e.status != http.HTTPStatus.NOT_FOUND:
            raise
    
    tensorboard_spec = textwrap.dedent(f'''\
            apiVersion: tensorboard.kubeflow.org/v1alpha1
            kind: Tensorboard
            metadata:
              name: "{tensorboard_name}"
              namespace: "{namespace}"
              ownerReferences:
                - apiVersion: v1
                  kind: Workflow
                  name: "{{workflow.name}}"
                  uid: "{{workflow.uid}}"
            spec:
              logspath: "pvc://{pvc_name}/{pvc_path}"
            '''
    )
    
    logger.info(tensorboard_spec)

    custom_object_api.create_namespaced_custom_object(
        group="tensorboard.kubeflow.org",
        version="v1alpha1",
        plural="tensorboards",
        namespace=namespace,
        body=yaml.safe_load(tensorboard_spec),
        pretty=True)

    tensorboard_watch = watch.Watch()
    try:
        for tensorboard_event in tensorboard_watch.stream(
            custom_object_api.list_namespaced_custom_object,
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            field_selector=f"metadata.name={tensorboard_name}",
            timeout_seconds=0,
        ):

            logger.info(f"tensorboard_event: {json.dumps(tensorboard_event, indent=2)}")

            if tensorboard_event["type"]=="DELETED":
                raise RuntimeError("The tensorboard was deleted!")

            tensorboard = tensorboard_event["object"]

            if "status" not in tensorboard:
                continue

            deployment_state = "Progressing"
            if "conditions" in tensorboard["status"]:
                deployment_state = tensorboard["status"]["conditions"][-1][
                    "deploymentState"
                ]

            if deployment_state == "Progressing":
                logger.info("Tensorboard deployment is progressing...")
            elif deployment_state == "Available":
                logger.info("Tensorboard deployment is Available.")
                break
            elif deployment_state == "ReplicaFailure":
                raise RuntimeError("Tensorboard deployment failed with a ReplicaFailure!")
            else:
                raise RuntimeError(f"Unknown deployment state: {deployment_state}")
    finally:
        tensorboard_watch.stop()

    button_style =(
        "align-items: center; "
        "appearance: none; "
        "background-color: rgb(26, 115, 232); "
        "border: 0px none rgb(255, 255, 255); "
        "border-radius: 3px; "
        "box-sizing: border-box; "
        "color: rgb(255, 255, 255); "
        "cursor: pointer; "
        "display: inline-flex; " 
        "font-family: 'Google Sans', 'Helvetica Neue', sans-serif; "
        "font-size: 14px; "
        "font-stretch: 100%; "
        "font-style: normal; font-weight: 700; "
        "justify-content: center; "
        "letter-spacing: normal; "
        "line-height: 24.5px; "
        "margin: 0px 10px 2px 0px; "
        "min-height: 25px; "
        "min-width: 64px; "
        "padding: 2px 6px 2px 6px; "
        "position: relative; "
        "tab-size: 4; "
        "text-align: center; "
        "text-indent: 0px; "
        "text-rendering: auto; "
        "text-shadow: none; "
        "text-size-adjust: 100%; "
        "text-transform: none; "
        "user-select: none; "
        "vertical-align: middle; "
        "word-spacing: 0px; "
        "writing-mode: horizontal-tb;"
    )

    # See: https://github.com/kubeflow/kubeflow/blob/master/components/crud-web-apps/tensorboards/frontend/src/app/pages/index/index.component.ts
    # window.open(`/tensorboard/${tensorboard.namespace}/${tensorboard.name}/`);
    ui_address = f"/tensorboard/{namespace}/{tensorboard_name}/#scalars"

    markdown = textwrap.dedent(
        f'''\
        # Tensorboard
        - <a href="{ui_address}" style="{button_style}" target="_blank">Connect</a>
        - <a href="/_/tensorboards/" style="{button_style}" target="_blank">Manage all</a>
        '''
    )

    markdown_output = {
        "type": "markdown",
        "storage": "inline",
        "source": markdown,
    }

    ui_metadata = {"outputs": [markdown_output]}
    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(ui_metadata, metadata_file)
   
    logging.info("Finished.")


configure_tensorboard_comp = kfp.components.create_component_from_func(
    func=configure_tensorboard, base_image=BASE_IMAGE
)

In [4]:
def load_df_from_db2(table_name: str,
                     data_frame_pkl: OutputPath(str)):
    import warnings
    import ibm_db
    import ibm_db_dbi
    import os
    import json
    import pandas as pd
    import pickle
    from typing import Dict, Any
    
    def assign_categories_to_df(df: pd.DataFrame, column_info: Dict[str, any]) -> None:
        for col_name, levels in column_info["label_columns"].items():
            if col_name in df.columns:
                ctype = pd.CategoricalDtype(categories=levels, ordered=False)
                df[col_name] = df[col_name].astype(ctype)

    def df_from_sql(
        name: str,
        conn: ibm_db.IBM_DBConnection,
        column_info: Dict[str, Any],
    ) -> pd.DataFrame:
        sql_safe_name = name.replace('"', "")

        rStmtColsSql = ",".join([f'"{col}"' for col in column_info["columns"]])
        rSql = f'SELECT {rStmtColsSql} FROM "{sql_safe_name}"'

        read_conn = ibm_db_dbi.Connection(conn)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="pandas only support SQLAlchemy")
            df = pd.read_sql(rSql, read_conn)

        assign_categories_to_df(df, column_info)
        return df
    
    conn_str = (
    "DRIVER={IBM DB2 ODBC DRIVER};"
    f"DATABASE=BLUDB;HOSTNAME={os.environ['db2_host']};PORT={os.environ['db2_port']};PROTOCOL=TCPIP;UID={os.environ['db2_user']};Pwd={os.environ['db2_pwd']};SECURITY=SSL;"
    )
        
    conn = ibm_db.connect(conn_str, "", "")

    column_info = json.loads(os.environ["COLUMNS"])
    df = df_from_sql(table_name, conn, column_info)
    df.to_pickle(data_frame_pkl)


load_df_from_db2_comp = kfp.components.create_component_from_func(
    func=load_df_from_db2, base_image=BASE_IMAGE
)

In [5]:
def data_quality_report(df: InputPath(str),
                        features: List[str],
                        mlpipeline_ui_metadata_path: OutputPath(str),
                        output_report: OutputPath(str),
                        target: str = 'Risk'):
    from evidently.metric_preset import DataQualityPreset
    from evidently.report import Report
    from evidently import ColumnMapping
    import pandas as pd
    import os
    from pathlib import Path
    import json

    
    dataset = pd.read_pickle(df)
    column_info = json.loads(os.environ["COLUMNS"])

    column_mapping = ColumnMapping()
    column_mapping.target = target
    column_mapping.task = "classification"
    feature_set = set(features)
    column_mapping.numerical_features = [
        c
        for c in column_info["int_columns"]
        if c in feature_set
    ]
    column_mapping.categorical_features = [
        c
        for c in column_info["label_columns"]
        if c in feature_set
    ]

    report = Report(
        metrics=[
            DataQualityPreset(),
        ]
    )

    report.run(
        reference_data=None,
        current_data=dataset,
        column_mapping=column_mapping,
    )

    Path(output_report).parent.mkdir(parents=True, exist_ok=True)
    report.save_html(output_report)
    html_content = open(output_report, "r").read()
    metadata = {
        "outputs": [
            {
                "type": "web-app",
                "storage": "inline",
                "source": html_content,
            }
        ]
    }

    with open(mlpipeline_ui_metadata_path, "w") as f:
        json.dump(metadata, f)
        
data_quality_report_comp = kfp.components.create_component_from_func(
    func=data_quality_report, base_image=BASE_IMAGE
)

In [6]:
def fit_preprocessor(
    training_df: InputPath(str),
    preprocessor_pkl: OutputPath(str),
    features: List[str],
):
    import pandas as pd
    import json
    import joblib
    import os

    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.pipeline import Pipeline

    feature_set = set(features)
    column_info = json.loads(os.environ["COLUMNS"])

    ohe_labels = [
        (
            "ohe_" + label,
            OneHotEncoder(
                handle_unknown="ignore", sparse_output=False, categories=[levels]
            ),
            [label],
        )
        for label, levels in column_info["label_columns"].items()
        if label in feature_set
    ]

    int_cols = [
        (
            "passthrough",
            "passthrough",
            [col for col in column_info["int_columns"] if col in feature_set],
        )
    ]

    pipe = Pipeline(
        steps=[
            ("Preprocess", ColumnTransformer(ohe_labels + int_cols, remainder="drop")),
        ]
    )

    print(pipe)
    train = pd.read_pickle(training_df)
    print(train.dtypes)
    pipe.fit(train)
    joblib.dump(pipe, preprocessor_pkl)


fit_preprocessor_comp = kfp.components.create_component_from_func(
    func=fit_preprocessor, base_image=BASE_IMAGE
)

In [7]:
def train(
    training_df: InputPath(str),
    preprocessor: InputPath(str),
    model: OutputPath(str),
    target_processing_config: OutputPath(str),
    mlpipeline_ui_metadata_path: OutputPath(str),
    target: str = "Risk",
    tensorboard_dir: str = None,
):
    import pandas as pd
    import json
    import joblib
    import tensorflow as tf
    from keras import Sequential
    from keras.layers import Dense, Dropout, BatchNormalization, Input
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
    from sklearn.metrics import precision_recall_curve
    import numpy as np
    import os
    from sklearn.metrics import PrecisionRecallDisplay
    import base64

    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.DEBUG)
    target_processing_config_dict = {
        "threshold" : 0.5,
        "target_names" : {0: "No Risk", 1: "Risk"}
    }

    def get_tf_model(num_features: int) -> Tuple[tf.keras.Model, List[tf.keras.callbacks.Callback]]:

        tf_model = Sequential(
            [
                Input(shape=(num_features,)),
                BatchNormalization(),
                Dense(35, activation="sigmoid", name="layer1"),
                BatchNormalization(),
                Dropout(0.3, name="dropout1"),
                Dense(35, activation="sigmoid", name="layer2"),
                BatchNormalization(),
                Dropout(0.3, name="dropout2"),
                Dense(35, activation="sigmoid", name="layer3"),
                Dropout(0.3, name="dropout3"),
                Dense(
                    1,
                    activation="sigmoid",
                    name="output",
                ),
            ]
        )

        tf_model.compile(optimizer="adam", loss="binary_crossentropy")

        callbacks = [
            EarlyStopping(
                monitor="val_loss",
                patience=20,
                verbose=0,
                mode="min",
                restore_best_weights=True,
            ),
            ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.1,
                patience=10,
                verbose=1,
                min_delta=0.0001,
                mode="min",
            )
        ]
        print(f"constructing tensorborad with log dir {tensorboard_dir}...")
        if tensorboard_dir:
            callbacks.append(TensorBoard(
                log_dir=tensorboard_dir
            ))

        return tf_model, callbacks

    print("loading training data...")
    train = pd.read_pickle(training_df)
    print("loading preprocessor...")
    preprocessor = joblib.load(preprocessor)

    X = tf.convert_to_tensor(preprocessor.transform(train))
    y = tf.convert_to_tensor(
        train.loc[:, target].apply(lambda v: 1 if v == target_processing_config_dict["target_names"][1] else 0)
    )
    print("obtaining model....")
    tf_model, callbacks = get_tf_model(num_features=X.shape[1])
    print("Training...")
    tf_model.fit(
        X,
        y,
        validation_split=0.2,
        epochs=500,
        callbacks=callbacks,
        class_weight={0: 1, 1: 2},
    )
    
    # calculate best threshold of highest f1 score
    predictions = tf_model.predict(X)
    precision, recall, thresholds = precision_recall_curve(
        y_true=y.numpy(), probas_pred=predictions.flatten()
    )
    f1s = 2 * (precision * recall) / (precision + recall)
    threshold_index = np.argmax(f1s)
    target_processing_config_dict["threshold"] = float(thresholds[threshold_index])
    
    # Save model and threshold config
    tf_model.save(model, save_format="h5")
    with open(target_processing_config, "w") as f:
        json.dump(target_processing_config_dict, f)
        
    
    plt = PrecisionRecallDisplay.from_predictions(
        y_true=y.numpy(), y_pred=predictions.flatten(),
    )
    plt.ax_.plot(recall[threshold_index], precision[threshold_index], 
                     marker="o", markersize=10, markeredgecolor="black", markerfacecolor="red")
    plt.ax_.text(recall[threshold_index] + .03, precision[threshold_index] + .03,
                  (f"({recall[threshold_index]:.3f},{precision[threshold_index]:.3f})\n" +
                   f"threshold={thresholds[threshold_index]:.3f}\n" +
                   f"f1={f1s[threshold_index]:.3f}")
                )
    plt.ax_.set_title("Precision Recall - Risk (Training)")
    #plt.ax_.set_xticks([0,.25, .5, 1] + [recall[threshold_index]])
    #plt.ax_.set_yticks([0,.25,.5,1] + [precision[threshold_index]])
    plt.figure_.savefig("pr.jpg")
    with open("pr.jpg", "rb") as f:
        jpg = base64.b64encode(f.read())
    html = f'<img src="data:image/jpg;base64,{jpg.decode("utf-8")}"/>'
    metadata = {"outputs": [{"type": "markdown", "storage": "inline", "source": html}]}
    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(metadata, metadata_file)

train_comp = kfp.components.create_component_from_func(
    func=train, base_image=BASE_IMAGE, packages_to_install=["minio"]
)

In [8]:
def evaluate(
    df: InputPath(str),
    preprocessor: InputPath(str),
    model: InputPath(str),
    target_processing_config: InputPath(str),
    output_report: OutputPath(str),
    mlpipeline_ui_metadata_path: OutputPath(str),
    target="Risk",
):
    import pandas as pd
    import joblib
    import tensorflow as tf
    import json
    from evidently.metric_preset import ClassificationPreset
    from evidently.report import Report
    from evidently import ColumnMapping
    import os
    from pathlib import Path

    dataset = pd.read_pickle(df)
    preprocessor = joblib.load(preprocessor)
    tf_model = tf.keras.models.load_model(model)
    with open(target_processing_config, "r") as f:
        target_processing_config_dict = json.load(f)
        target_processing_config_dict["target_names"] = {int(k):v for k,v in target_processing_config_dict["target_names"].items()}

    column_info = json.loads(os.environ["COLUMNS"])

    X = preprocessor.transform(dataset)
    y_prob = tf_model.predict(X).flatten()

    dataset["Prediction"] = pd.Series(y_prob).apply(
        lambda p: 1 if p > target_processing_config_dict["threshold"] else 0
    )
    dataset["Actual"] = dataset.loc[:, target].apply(
        lambda v: 1 if v == target_processing_config_dict["target_names"][1] else 0
    )

    column_mapping = ColumnMapping()
    column_mapping.target_names = target_processing_config_dict["target_names"]
    column_mapping.target = "Actual"
    column_mapping.prediction = "Prediction"
    column_mapping.task = "classification"
    column_mapping.numerical_features = [
        c
        for c in column_info["int_columns"]
        if c in set(preprocessor.feature_names_in_)
    ]
    column_mapping.categorical_features = [
        c
        for c in column_info["label_columns"]
        if c in set(preprocessor.feature_names_in_)
    ]

    report = Report(
        metrics=[
            ClassificationPreset(),
        ]
    )

    report.run(
        reference_data=None,
        current_data=dataset,
        column_mapping=column_mapping,
    )

    Path(output_report).parent.mkdir(parents=True, exist_ok=True)
    report.save_html(output_report)
    html_content = open(output_report, "r").read()
    metadata = {
        "outputs": [
            {
                "type": "web-app",
                "storage": "inline",
                "source": html_content,
            }
        ]
    }

    with open(mlpipeline_ui_metadata_path, "w") as f:
        json.dump(metadata, f)

eval_comp = kfp.components.create_component_from_func(
    func=evaluate, base_image=BASE_IMAGE
)

In [9]:
def train_explainer(train_df: InputPath(str),
                    preprocessor: InputPath(str),
                    tf_model: InputPath(str),
                    target_processing_config: InputPath(str),
                    explainer_dll: OutputPath(str)):
    from alibi.explainers.anchors import anchor_tabular
    from alibi.utils import gen_category_map
    import joblib
    from sklearn.pipeline import Pipeline
    import re
    import numpy as np
    import dill
    from functools import partial
    from typing import Tuple, List, Dict
    import json
    import os
    import pandas as pd
    import tensorflow as tf
    
    def generate_category_map(preprocessor_pipeline: Pipeline) -> Tuple[List[str], Dict[int, List[str]]]:
        features = []
        seen_features = set()
        category_map = dict()

        for out_col_name in preprocessor_pipeline.get_feature_names_out():
            parts = re.search("(.*)__([^_]+)(_([A-Za-z0-9_-]+))?", out_col_name)
            if parts:
                if parts.group(2) not in seen_features:
                    features.append(parts.group(2))
                    seen_features.add(parts.group(2))

                if parts.group(1) == "categorical" or parts.group(1).startswith("ohe_"):
                    levels = category_map.get(len(features) - 1, [])
                    levels.append(parts.group(4))
                    category_map[len(features) - 1] = levels
            else:
                raise ValueError("Could not parse column " + out_col_name)

        return features, category_map


    with open(target_processing_config, "r") as f:
        target_processing_config_dict = json.load(f)
    column_info = json.loads(os.environ["COLUMNS"])
        
    model = tf.keras.models.load_model(tf_model)
    threshold:float = float(target_processing_config_dict["threshold"])
    def predict(X: np.ndarray) -> np.ndarray:
        probs = model.predict(X).flatten()
        predictions = pd.Series(probs).apply(lambda p: 1 if p > threshold else 0).to_numpy()
        return predictions
    
    preprocessor_pipeline = joblib.load(preprocessor)
    features, category_map = generate_category_map(preprocessor_pipeline)
    
    dataset = pd.read_pickle(train_df)
    X = preprocessor_pipeline.transform(dataset)
    
    explainer = anchor_tabular.AnchorTabular(
        predict, features, categorical_names=category_map, ohe=True, seed=42
    )
    explainer.fit(X, disc_prec=[10,25,33,50,66,75,90])
    
    explainer.predict_fn = None  # Clear explainer predict_fn as its a lambda and will be reset when loaded
    with open(explainer_dll, "wb") as f:
        dill.dump(explainer, f)

train_explainer_comp = kfp.components.create_component_from_func(
    func=train_explainer, base_image=BASE_IMAGE
)

In [10]:
def convert_model_to_onnx(tf_model: InputPath(str),
                          onnx_model: OutputPath(str)):
    import tf2onnx
    import tensorflow as tf
    import onnx
    
    keras_model = tf.keras.models.load_model(tf_model)    
    converted_model, _ = tf2onnx.convert.from_keras(keras_model)
    onnx.save_model(converted_model, onnx_model)

convert_model_to_onnx_comp = kfp.components.create_component_from_func(
    func=convert_model_to_onnx, base_image=BASE_IMAGE
)

In [11]:
def upload_artifacts(
    onnx_model: InputPath(str),
    preprocessor: InputPath(str),
    explainer: InputPath(str),
    archive_name: str,
    minio_url: str = "minio-service.kubeflow:9000",
    version: str = "1"
) -> NamedTuple("UploadOutput", [("s3_address", str)]):
    """Uploads a model file to MinIO artifact store."""

    from collections import namedtuple
    import logging
    from minio import Minio
    import sys
    import tarfile
    import os

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()

    ARCHIVE_FILE = f"/tmp/{archive_name}"
    with tarfile.open(ARCHIVE_FILE, "w") as f:
        f.add(onnx_model, arcname="model.onnx")
        f.add(preprocessor, arcname="preprocessor.joblib")
        f.add(explainer, arcname="explainer.dll")

    minio_client = Minio(
            minio_url, 
            access_key=os.environ["MINIO_ID"], 
            secret_key=os.environ["MINIO_PWD"], secure=False
        )

    # Create export bucket if it does not yet exist
    export_bucket="{{workflow.namespace}}"
    existing_bucket = next(filter(lambda bucket: bucket.name == export_bucket, minio_client.list_buckets()), None)

    if not existing_bucket:
        logger.info(f"Creating bucket '{export_bucket}'...")
        minio_client.make_bucket(bucket_name=export_bucket)

    path = f"tar/{version}/{archive_name}"
    s3_address = f"s3://{minio_url}/{export_bucket}/{path}"

    logger.info(f"Saving onnx file to MinIO (s3 address: {s3_address})...")
    minio_client.fput_object(
        bucket_name=export_bucket,  # bucket name in Minio
        object_name=path,  # file name in bucket of Minio 
        file_path=ARCHIVE_FILE,  # file path / name in local system
    )

    logger.info("Finished.")
    out_tuple = namedtuple("UploadOutput", ["s3_address"])
    return out_tuple(s3_address)


upload_artifacts_comp = kfp.components.create_component_from_func(
    func=upload_artifacts, base_image=BASE_IMAGE
)

In [12]:
from kubernetes.client import ( V1PersistentVolumeClaimVolumeSource, V1Volume, V1VolumeMount)
@dsl.pipeline(
    name="Credit Risk",
    description="An example pipeline that builds and deploys a credit risk model",
)
def credit_model_pipeline():
    def env_var_from_secret(env_var_name: str, secret_name: str, secret_key: str) -> V1EnvVar:
        return V1EnvVar(name=env_var_name,
                                     value_from=V1EnvVarSource(
                                         secret_key_ref=V1SecretKeySelector(
                                             name=secret_name,
                                             key=secret_key
                                         )
                                     )
                                    )
    def add_db2_connection_secrets(pipeline_task) -> None:
        pipeline_task.container.add_env_variable(env_var_from_secret("db2_host", "db2-credentials", "host"))
        pipeline_task.container.add_env_variable(env_var_from_secret("db2_user", "db2-credentials", "username"))
        pipeline_task.container.add_env_variable(env_var_from_secret("db2_pwd", "db2-credentials", "password"))
        pipeline_task.container.add_env_variable(env_var_from_secret("db2_port", "db2-credentials", "port"))

    feature_columns = [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker",
        ]
    
    load_training_data_task = load_df_from_db2_comp(table_name="TRAIN")
    load_training_data_task.set_display_name("Load_Training_Data_From_DB2")
    add_db2_connection_secrets(load_training_data_task)

    data_quality_report_comp(df=load_training_data_task.outputs["data_frame_pkl"],
                             features=feature_columns)
    
    load_test_data_task = load_df_from_db2_comp(table_name="TEST")
    load_test_data_task.set_display_name("Load_Test_Data_From_DB2")
    add_db2_connection_secrets(load_test_data_task)

    fit_preprocessor_task = fit_preprocessor_comp(
        training_df=load_training_data_task.outputs["data_frame_pkl"],
        features=feature_columns
    )

    create_tensorboard_volume = dsl.VolumeOp(
        name=f"Create PVC for tensorboard",
        resource_name="tensorboard",
        modes=dsl.VOLUME_MODE_RWM,
        size="4G",
        set_owner_reference=True,
    )
    create_tensorboard_volume.add_pod_annotation(
            name="pipelines.kubeflow.org/max_cache_staleness", value="P0D"
        )
        
    configure_tensorboard_task = configure_tensorboard_comp(
        pvc_name=create_tensorboard_volume.volume.persistent_volume_claim.claim_name
    )
    
    train_model_task = train_comp(
        training_df=load_training_data_task.outputs["data_frame_pkl"],
        preprocessor=fit_preprocessor_task.outputs["preprocessor_pkl"],
        target="Risk",
        tensorboard_dir="/tensorboard",
    )
    train_model_task.after(configure_tensorboard_task)
    train_model_task.add_pvolumes({"/tensorboard": create_tensorboard_volume.volume})

    evaluate_model_task = eval_comp(
        load_test_data_task.outputs["data_frame_pkl"],
        preprocessor=fit_preprocessor_task.outputs["preprocessor_pkl"],
        model=train_model_task.outputs["model"],
        target_processing_config=train_model_task.outputs["target_processing_config"],
        target="Risk"
    )
    
    train_explainer_task = train_explainer_comp(train_df=load_training_data_task.outputs["data_frame_pkl"],
                                                preprocessor=fit_preprocessor_task.outputs["preprocessor_pkl"],
                                                tf_model=train_model_task.outputs["model"],
                                                target_processing_config=train_model_task.outputs["target_processing_config"])
    
    convert_model_to_onnx_task = convert_model_to_onnx_comp(tf_model=train_model_task.outputs["model"])
    
    upload_artifacts_task = upload_artifacts_comp(
        onnx_model=convert_model_to_onnx_task.outputs["onnx_model"],
        preprocessor=fit_preprocessor_task.outputs["preprocessor_pkl"],
        explainer=train_explainer_task.outputs["explainer_dll"],
        archive_name="credit-risk.tar"
    )
    upload_artifacts_task.container.add_env_variable(env_var_from_secret("MINIO_ID", "mlpipeline-minio-artifact", "accesskey"))
    upload_artifacts_task.container.add_env_variable(env_var_from_secret("MINIO_PWD", "mlpipeline-minio-artifact", "secretkey"))


In [13]:
def delete_pipeline(pipeline_name: str):
    """Delete's a pipeline with the specified name"""

    client = kfp.Client()
    existing_pipelines = client.list_pipelines(page_size=999).pipelines
    matches = (
        [ep.id for ep in existing_pipelines if ep.name == pipeline_name]
        if existing_pipelines
        else []
    )
    for id in matches:
        client.delete_pipeline(id)


In [14]:
def get_experiment_id(experiment_name: str) -> str:
    """Returns the id for the experiment, creating the experiment if needed"""
    client = kfp.Client()
    existing_experiments = client.list_experiments(page_size=999).experiments
    matches = (
        [ex.id for ex in existing_experiments if ex.name == experiment_name]
        if existing_experiments
        else []
    )

    if matches:
        return matches[0]

    exp = client.create_experiment(experiment_name)
    return exp.id


In [15]:
pipeline_conf = kfp.dsl.PipelineConf()

def provide_column_info_transformer(op: dsl.ContainerOp):
    
    if isinstance(op, dsl.ContainerOp):
        op.container.add_env_variable(
            V1EnvVar(name="COLUMNS",
                    value_from=V1EnvVarSource(
                                         config_map_key_ref=V1ConfigMapKeySelector(
                                             name="credit-risk-columns",
                                             key="columns"
                                         )
                                     )
                    )
        )
                            

pipeline_conf.add_op_transformer(provide_column_info_transformer)



In [16]:
PIPELINE_NAME = "Build_Credit_Risk_Model"
# Pipeline names need to be unique, so before we upload,
# check for and delete any pipeline with the same name
delete_pipeline(PIPELINE_NAME)

        
kfp.compiler.Compiler().compile(
    pipeline_func=credit_model_pipeline,
    package_path=f"{PIPELINE_NAME}.yaml",
    pipeline_conf=pipeline_conf,
)

# upload
client = kfp.Client()
uploaded_pipeline = client.upload_pipeline(f"{PIPELINE_NAME}.yaml", PIPELINE_NAME)


In [17]:
run = client.run_pipeline(
    experiment_id=get_experiment_id("credit-risk"),
    job_name="credit-risk",
    pipeline_id=uploaded_pipeline.id,
)

In [18]:
TWENTY_MIN = 20 * 60
result = client.wait_for_run_completion(run.id, timeout=TWENTY_MIN)
{
    "status": result.run.status,
    "error": result.run.error,
    "time": str(result.run.finished_at - result.run.created_at),
    "metrics": result.run.metrics,
}


{'status': 'Succeeded', 'error': None, 'time': '0:04:27', 'metrics': None}